In [13]:
import pandas as pd

In [14]:
resp = pd.read_csv('/content/amostra_recurso.csv')

In [15]:
resp.head()

,recurso,tema,tese,num_tema_cadastrado,sugerido_1,sugerido_2,sugerido_3,sugerido_4,sugerido_5,sugerido_6,probabilidade_1,probabilidade_2,probabilidade_3,probabilidade_4,probabilidade_5,probabilidade_6,sugestao_adotada
0,EXCELENTÍSSIMO SENHOR VICE-PRESIDENTE DO EGRÉG...,Definir se a liquidação prévia do julgado é re...,NaN,1169,1056,1130.0,1178.0,1033.0,1086.0,NaN,37,24,13,13,13,0,0
1,\r\nADVOCACIA-GERAL DA UNIÃO\r\nPROCURADORIA-G...,Fornecimento de Equipamento de Proteção Indivi...,I - O direito à aposentadoria especial pressup...,555,1083,694.0,534.0,555.0,NaN,NaN,35,30,31,4,0,0,4
2,\r\nADVOCACIA-GERAL DA UNIÃO\r\nPROCURADORIA-G...,Definir se é possível ao magistrado fixar crit...,NaN,7,1090,534.0,1081.0,1083.0,694.0,NaN,33,23,22,11,11,0,0
3,PROCURADORIA-GERAL DA FAZENDA NACIONAL\r\n\r\n...,Inclusão da COFINS e da contribuição ao PIS em...,NaN,1067,1125,1182.0,NaN,NaN,NaN,NaN,87,13,0,0,0,0,0
4,ADVOCACIA-GERAL DA UNIÃƒOPROCURADORIA-GERAL FE...,Definir se a demanda previdenciária cujo valor...,NaN,1081,1081,NaN,NaN,NaN,NaN,NaN,100,0,0,0,0,0,1


**Modelo Legal-BERTimbau**
[huggingface](https://huggingface.co/rufimelo/Legal-BERTimbau-base)

@inproceedings{souza2020bertimbau,
  author    = {F{\'a}bio Souza and
               Rodrigo Nogueira and
               Roberto Lotufo},
  title     = {{BERT}imbau: pretrained {BERT} models for {B}razilian {P}ortuguese},
  booktitle = {9th Brazilian Conference on Intelligent Systems, {BRACIS}, Rio Grande do Sul, Brazil, October 20-23 (to appear)},
  year      = {2020}
}


In [25]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.2 MB/s eta 0:00:00


In [37]:
import  torch
from  transformers  import  AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("rufimelo/Legal-BERTimbau-base")
model = AutoModel.from_pretrained('rufimelo/Legal-BERTimbau-base')

def ler_recurso(dataframe, linha, coluna):
    texto = dataframe.at[linha, coluna]
    return texto

conteudo = ler_recurso(resp,1,'recurso')
input_ids = tokenizer.encode(conteudo, return_tensors='pt')
#Limitacao de tamanho do texto para usar o modelo , para teste foi limitada a dimensao do tensor em 512
input_ids512 = input_ids[:, :512]

with  torch.no_grad():
    outs = model(input_ids512)
    encoded = outs[0][0, 1:-1]



Some weights of BertModel were not initialized from the model checkpoint at rufimelo/Legal-BERTimbau-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
print(encoded)

tensor([[ 0.3504, -0.6196,  0.8408,  ..., -0.6311, -0.0492, -0.2174],
        [-0.1156, -0.3521,  0.5572,  ..., -0.4350, -0.4658, -0.0934],
        [-0.1086, -0.3734,  0.2686,  ...,  0.3604,  0.0313, -0.3450],
        ...,
        [ 0.0545, -0.3944,  0.3708,  ..., -0.8043, -0.4407,  0.5829],
        [ 0.3855, -0.1784, -0.4938,  ..., -0.1290, -0.0845,  0.1714],
        [ 0.1156,  0.0647, -0.5313,  ..., -0.6083,  0.1171, -0.4277]])


In [32]:
print("Shape:", encoded.shape)
print("Número de dimensões:", encoded.dim())
print("Tipo de dado:", encoded.dtype)
print("Dispositivo (CPU/GPU):", encoded.device)
print("Número total de elementos:", encoded.numel())

Shape: torch.Size([8, 768])
Número de dimensões: 2
Tipo de dado: torch.float32
Dispositivo (CPU/GPU): cpu
Número total de elementos: 6144
